## Knapsack problem

### Create knapsack problem

In [2]:
from QHyper.problems.knapsack import KnapsackProblem

# Create knapsack with 3 items: 2 with weights 1 and value 2 - (1, 2),
# and one with weight 1 and value 1 - (1, 1)
knapsack = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2),(1, 1)])
print(knapsack.knapsack.items)

[Item(weight=1, value=2), Item(weight=1, value=2), Item(weight=1, value=1)]


In [3]:
print(f"Variables used to describe objective function"
      f"and constraints: {knapsack.variables}")
print(f"Objective function: {knapsack.objective_function}")
print("Constraints:")
for constraint in knapsack.constraints:
    print(f"    {constraint}")

Variables used to describe objective functionand constraints: (x0, x1, x2, x3, x4)
Objective function: -2*x0 - 2*x1 - x2
Constraints:
    -x3 - x4 + 1
    -x0 - x1 - x2 + x3 + 2*x4


### Use VQA to solve knapsack problem

In [4]:
import numpy as np

from QHyper.solvers.vqa.base import VQA

In [6]:
# Simple quantum circuit without optimzers will be used to test the results
# WF-QAOA is choosen becasue this PQC has most suitable evaluation function

tester_config = {
    "pqc": {
        "type": "wfqaoa",
        "layers": 5,
    }
}

tester = VQA(knapsack, config=tester_config)

In [8]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)

solver_config = {
    'optimizer': {
        'type': 'scipy',
        'maxfun': 200,
    },
    "pqc": {
        "type": "hqaoa",
        "layers": 5,
    },
}
vqa = VQA(knapsack, config=solver_config)

# 2. Providing actual isntance of each class like VQA and Optimizer
from QHyper.solvers.vqa.pqc.h_qaoa import HQAOA
from QHyper.optimizers import ScipyOptimizer

vqa = VQA(knapsack, HQAOA(layers=5), ScipyOptimizer(maxfun=200))

In [12]:
# Run VQA with provided initial parameters

params_cofing = {
    'angles': [[0.5]*5, [1]*5],
    'hyper_args': [1, 2.5, 2.5],
}
best_params = vqa.solve(params_cofing)
print(f"Best params: {best_params}")

Best params: {'angles': array([2.83711847e+00, 1.97940203e-01, 1.07858283e+00, 1.63047414e+00,
       4.04555732e+00, 1.35028794e+00, 5.49151498e-01, 8.21588171e-01,
       9.77277636e-04, 2.20975660e+00]), 'hyper_args': array([1.31189481, 2.95215395, 3.76936221])}


In [13]:
# Here created ealier tester will be used to evaluate results

best_results = tester.evaluate(best_params)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params['angles']},\n"
      f"and params used for hyper optimizer: {best_params['hyper_args']}")

Best results: -0.9657474465012011
Params used for optimizer:
[2.83711847e+00 1.97940203e-01 1.07858283e+00 1.63047414e+00
 4.04555732e+00 1.35028794e+00 5.49151498e-01 8.21588171e-01
 9.77277636e-04 2.20975660e+00],
and params used for hyper optimizer: [1.31189481 2.95215395 3.76936221]


#### Using hyper optimizers

In [18]:
# Additionally other optimizer can be used to tune some parameters, in below
# example, Random optimzier will change weights (hyper_args) and choose ones
# that gives the best results after runnign 200 iteration of scipy minimizer

from QHyper.optimizers.random import Random

solver_config = {
    'pqc': {
        'type': 'hqaoa',
        'layers': 5,
    },
    'optimizer': {
        'type': 'scipy',
        'maxfun': 200
    }
}
params_cofing = {
    "angles": [[0.5] * 5, [1] * 5],
    "hyper_args": [1, 2.5, 2.5],
}
vqa = VQA(knapsack, config=solver_config)

random = Random(processes=5, number_of_samples=100, bounds=[[1, 10], [1, 10], [1, 10]])
best_params_h = vqa.solve(params_cofing, random)

100%|██████████| 100/100 [06:24<00:00,  3.85s/it]


In [19]:
# Again, final results will be evaluated with tester

best_results = tester.evaluate(best_params_h)
print(f"Best results: {best_results}")
print(f"Params used for optimizer:\n{best_params_h['angles']},\n"
      f"and params used for hyper optimizer: {best_params['hyper_args']}")

Best results: -2.0780820817153285
Params used for optimizer:
[0.57802023 0.44967026 0.39974621 0.61983699 0.52300296 0.99195499
 0.97691219 1.21249674 1.31225575 0.99615924],
and params used for hyper optimizer: [1.31189481 2.95215395 3.76936221]
